# Eat Safe, Love

## Notebook Set Up

In [14]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [15]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [16]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [17]:
# review the collections in our database
collection_names = db.list_collection_names()
pprint(collection_names)

['establishments']


In [18]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [20]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
num_documents = establishments.count_documents(query)
print(f"Number of establishments with hygiene score of 20: {num_documents}")

# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments with hygiene score of 20: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('669f1366c35e63c2463396db'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishm

In [21]:
# Convert the result to a Pandas DataFrame
results = establishments.find(query)
df = pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
pprint(len(df))

# Display the first 10 rows of the DataFrame
pprint(df.head(10))

41
                        _id   FHRSID  ChangesByServerID  \
0  669f1366c35e63c2463396db   110681                  0   
1  669f1366c35e63c246339a5b   612039                  0   
2  669f1366c35e63c246339d68   730933                  0   
3  669f1366c35e63c246339f58   172735                  0   
4  669f1366c35e63c246339f64   172953                  0   
5  669f1366c35e63c24633a902   512854                  0   
6  669f1366c35e63c24633ab23  1537089                  0   
7  669f1366c35e63c24633c04a   155648                  0   
8  669f1366c35e63c24633c493  1012883                  0   
9  669f1366c35e63c24633cca6   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532              Golden Palace   
5    

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [22]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query ={
    "LocalAuthorityName": "London",
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
results = establishments.find(query)
num_documents = establishments.count_documents(query)
# Display the first document in the results using pprint
pprint(first_document)

{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('669f1366c35e63c2463396db'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'

In [28]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(first_document))

# Display the number of rows in the DataFrame
print(len(df))

# Display the first 10 rows of the DataFrame
pprint(df.head(10))

28
                          0
0                       _id
1                    FHRSID
2         ChangesByServerID
3  LocalAuthorityBusinessID
4              BusinessName
5              BusinessType
6            BusinessTypeID
7              AddressLine1
8              AddressLine2
9              AddressLine3


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [24]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384

query = {
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
    "RatingValue": 5
}

# Sort by hygiene score
sort = [("scores.Hygiene", 1)]  # 1 for ascending order, -1 for descending

# Limit number of results (optional)
limit = 10

# Print the results
results = establishments.find(query).sort(sort).limit(limit)
for result in results:
    pprint(result)

{'AddressLine1': '6 Lakedale Road',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'AddressLine4': 'Greenwich',
 'BusinessName': 'Greggs',
 'BusinessType': 'Takeaway/sandwich shop',
 'BusinessTypeID': 7844,
 'ChangesByServerID': 0,
 'Distance': 4646.987290353701,
 'FHRSID': 694482,
 'LocalAuthorityBusinessID': 'PI/000086551',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE18 1PP',
 'RatingDate': '2021-06-23T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('669f1366c35e63c24633ef7f'),
 'geocode': {'latitude': Decimal128('51.4871406555176'),
             'longitude': Decimal128('0.0912820026278496')},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/694482',
            'rel': 'self'}],
 'meta': 

In [31]:
# Convert result to Pandas DataFrame
df = pd.DataFrame(list(result))

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [30]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {"$match": {"scores.Hygiene": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
# Print the number of documents in the result
agg_result = list(establishments.aggregate(pipeline))

pprint(len(agg_result))

# Print the first 10 results
for result in agg_result[:10]:
    pprint(result)

56
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [29]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(list(result))

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print(df.head(10))


Number of rows in the DataFrame: 28
                          0
0                       _id
1                    FHRSID
2         ChangesByServerID
3  LocalAuthorityBusinessID
4              BusinessName
5              BusinessType
6            BusinessTypeID
7              AddressLine1
8              AddressLine2
9              AddressLine3
